In [1]:
import os
import random
import string
from fpdf import FPDF


In [64]:
class Generator:

    orientation_map = {
        1: "horizontal",
        2: "vertical",
        3: "diagonal"
    }

    alphabets = list(string.ascii_uppercase)

    def __init__(self, filename):
        self.grid_width = 0
        self.grid_height = 0
        self.words = []
        self.word_list_inserted = []
        self.filled_coordinates = []
        self.retries = 0

        self._read_file(filename)
        self._generate_grid()

    def _set_dimensions(self, width, height):
        self.grid_width = width
        self.grid_height = height
    
    def _set_words(self, words):
        self.words = words
    
    def _read_file(self, filename):
        with open(filename,"r") as f:
            words = f.readlines()
            for word in words:
                self.words.append(word.strip().upper())

    def _get_longest_word_len(self):
        max_len = 0
        for word in self.words:
            max_len = max(len(word), max_len) 
        return max_len

    def _char_count(self):
        char_count = 0
        for word in self.words:
            char_count += len(word)
        return char_count

    def _insert_word(self, word):
        """y x coord system due to list indexing but coded in x,y coord"""
        while self.retries < 100:
            # get orientation of insertion
            orientation = self.orientation_map[random.choice([1, 2, 3])]

            # get random initial coord
            # x_size = self.grid_width if orientation == "vertical" else self.grid_width - len(word)
            # y_size = self.grid_height if orientation == "horizontal" else self.grid_height - len(word)
            # x = random.randrange(x_size)
            # y = random.randrange(y_size)
            x = random.choice(range(self.grid_width))
            y = random.choice(range(self.grid_height))

            # small check for out of bounds
            if orientation == "horizontal":
                final_x = x + len(word)
                if final_x > self.grid_width:
                    self.retries += 1
                    break

            if orientation == "vertical":
                final_y = y + len(word)
                if final_y > self.grid_height:
                    self.retries += 1
                    break

            if orientation == "diagonal":
                final_x = x + len(word)
                final_y = y + len(word)
                if final_x > self.grid_width:
                    self.retries += 1
                    break
                if final_y > self.grid_height:
                    self.retries += 1
                    break


            # update the filled_coordinates and update grid
            coords = [x, y]
            temp_coords = []
            if orientation == "horizontal":
                for i in range(len(word)):
                    coords = [x + i, y]
                    if coords in self.filled_coordinates:
                        self.retries += 1
                        break
                    if coords[0] > self.grid_width:
                        self.retries += 1
                        break
                    self.grid[coords[1]][coords[0]] = word[i]
                    temp_coords.append(coords)
                    print(f"{word} last at {coords[0], coords[1]}, horizontal")

            if orientation == "vertical":
                for i in range(len(word)):
                    coords = [x, y + i]
                    if coords in self.filled_coordinates:
                        self.retries += 1
                        break
                    if coords[1] > self.grid_width:
                        self.retries += 1
                        break
                    self.grid[coords[1]][coords[0]] = word[i]
                    temp_coords.append(coords)
                    print(f"{word} last at {coords[0], coords[1]}, vertical")

            if orientation == "diagonal":
                for i in range(len(word)):
                    coords = [x + i, y + i]
                    if coords in self.filled_coordinates:
                        self.retries += 1
                        break
                    if coords[0] > self.grid_width:
                        self.retries += 1
                        break
                    if coords[1] > self.grid_width:
                        self.retries += 1
                        break
                    self.grid[coords[1]][coords[0]] = word[i]
                    temp_coords.append(coords)
                    print(f"{word} last at {coords[0], coords[1]}, diagonal")

            self.filled_coordinates.append(temp_coords)
            self.word_list_inserted.append(word)
            break



    
    def display_grid(self):
        print("\n".join(map(lambda row: " ".join(row), self.grid)))

    def _generate_grid(self):
        char_count = self._char_count()
        max_len = self._get_longest_word_len()
        valid = False
        scale_factor = 1.5

        while not valid:
            width = round(scale_factor * max_len)
            height = round(scale_factor * max_len)

            area = width * height

            if area > 2 * char_count:
                valid = True
            
            scale_factor += 0.1
        
        # set the dimensions 
        self._set_dimensions(width, height)

        # initialise a random grid
        self.grid = [[random.choice(string.ascii_uppercase) for i in range(0, width)] for j in range(0, height)]

        # insert words one by one
        for word in self.words:
            self._insert_word(word)
    
    def export_to_pdf(self, title, lines, grid_font):
        print(title)
        hints_per_row = 4
        
        pdf = FPDF(orientation = 'P',unit = 'mm', format='A4')

        gridx = 190
        gridy = 6

        # Add a page
        pdf.add_page()

        # set style and size of font
        # that you want in the pdf
        pdf.set_font("Arial", size = 25)
        title = title.upper()
        pdf.cell(gridx, gridy, txt = "", ln = 1)
        pdf.cell(gridx, gridy, txt = title, ln = 1, align = 'C')
        pdf.cell(gridx, gridy, txt = "", ln = 1)


        pdf.set_font("Courier", size = grid_font)
        pdf.cell(gridx, gridy, txt = "", ln = 1) 
        # create a cell
        for i in self.grid:
            x = "  ".join(i)
            pdf.cell(gridx, gridy, txt = x, ln = 1, align = 'C')

        pdf.cell(gridx, gridy, txt = "", ln = 1)    
        pdf.set_font("Arial", size = 25)
        pdf.cell(gridx, 10, txt = "Word List", ln = 1, align = 'C')

        pdf.set_font("Courier", size = 12)
        pdf.cell(gridx, gridy, txt = "", ln = 1)
        self.words = sorted(self.words)
        split_lists = [self.words[x:x+hints_per_row] for x in range(0, len(self.words), lines)]
        for i in split_lists:
            wordx = ", ".join(i)
            pdf.cell(gridx, grid_font, txt = wordx, ln = 1, align = 'C')   

        # save the pdf with name .pdf
        pdf.output("%s.pdf" %(title))  
        

# TODO: remove duplicates

g = Generator("word-list.txt")

TANK last at (8, 3), diagonal
TANK last at (9, 4), diagonal
TANK last at (10, 5), diagonal
TANK last at (11, 6), diagonal
JORDAN last at (3, 1), horizontal
JORDAN last at (4, 1), horizontal
JORDAN last at (5, 1), horizontal
JORDAN last at (6, 1), horizontal
JORDAN last at (7, 1), horizontal
JORDAN last at (8, 1), horizontal
EGYPT last at (8, 3), vertical
EGYPT last at (8, 4), vertical
EGYPT last at (8, 5), vertical
EGYPT last at (8, 6), vertical
EGYPT last at (8, 7), vertical
HAROLD last at (3, 6), horizontal
HAROLD last at (4, 6), horizontal
HAROLD last at (5, 6), horizontal
HAROLD last at (6, 6), horizontal
HAROLD last at (7, 6), horizontal
HAROLD last at (8, 6), horizontal
YOYO last at (0, 0), horizontal
YOYO last at (1, 0), horizontal
YOYO last at (2, 0), horizontal
YOYO last at (3, 0), horizontal


In [62]:
g = Generator("word-list.txt")

TANK last at (8, 3), diagonal
TANK last at (9, 4), diagonal
TANK last at (10, 5), diagonal
TANK last at (11, 6), diagonal
TANK last at (9, 3), diagonal
TANK last at (10, 4), diagonal
TANK last at (11, 5), diagonal
TANK last at (12, 6), diagonal
TANK last at (7, 10), vertical
TANK last at (7, 11), vertical
TANK last at (7, 12), vertical
TANK last at (7, 13), vertical
TANK last at (4, 11), diagonal
TANK last at (5, 12), diagonal
TANK last at (6, 13), diagonal
TANK last at (7, 14), diagonal
JORDAN last at (5, 0), horizontal
JORDAN last at (6, 0), horizontal
JORDAN last at (7, 0), horizontal
JORDAN last at (8, 0), horizontal
JORDAN last at (9, 0), horizontal
JORDAN last at (10, 0), horizontal
JORDAN last at (1, 8), horizontal
JORDAN last at (2, 8), horizontal
JORDAN last at (3, 8), horizontal
JORDAN last at (4, 8), horizontal
JORDAN last at (5, 8), horizontal
JORDAN last at (6, 8), horizontal
EGYPT last at (10, 12), horizontal
EGYPT last at (11, 12), horizontal
EGYPT last at (12, 12), hori

In [48]:
g.display_grid()

C Y M H X K Y V S C D P H P P
B B P J X R U U F X A K T X P
Z P K P Q N J E V T V G Q E K
G P N Z K N L N Y O Y O F Y Q
C G T U P N H M W E N N O A O
C V Q E U X Q D S O V Z K I Z
H A E V P C Z H L S B H X R M
P X O O H D Q I M A F E D B N
P Y B U H Q I H A Q E G T B K
D O H Z A I J O R D A Y X W G
H Y A S G T R J F J E P T P O
S O N J I M U Q H T R T C R O
Q O I F W B T S F A K F R M Q
F E O G G C R D W N I M B H B
N M W U N W U U I K M E H T F


In [65]:
g.export_to_pdf("word_search", 2, 20)

word_search


In [41]:
g.grid_width

16

In [39]:
for row in g.grid:
    assert len(row) == g.grid_width

In [66]:
x = [[1,2]]